Le LSTM est un type de réseau de neurones qui est particulièrement adapté pour travailler avec des données séquentielles,    
telles que des séries temporelles. Pour prédire les prix des vols, le LSTM peut être entraîné sur un ensemble de données historiques de prix de vols,     
ainsi que sur d'autres variables importantes telles que les dates, les heures de départ, les aéroports de départ et d'arrivée, etc.

Le LSTM peut alors utiliser ces données historiques pour apprendre les tendances et les modèles de fluctuation des prix des vols,    
et pour prédire les prix futurs des vols en fonction des données actuelles.

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Charger les données historiques de prix de vols depuis un fichier CSV
data = pd.read_csv('historical_flight_prices.csv')

# Prétraitement des données
scaler = MinMaxScaler()
data['Price'] = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

# Diviser les données en ensembles d'entraînement et de test
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_data = data[:train_size]
test_data = data[train_size:]

# Préparation des données pour l'apprentissage du modèle
def prepare_data(data, lookback):
    X, Y = [], []
    for i in range(len(data)-lookback-1):
        x = data[i:(i+lookback), 0]
        X.append(x)
        Y.append(data[i+lookback, 0])
    return np.array(X), np.array(Y)

lookback = 5
train_X, train_Y = prepare_data(train_data['Price'].values.reshape(-1, 1), lookback)
test_X, test_Y = prepare_data(test_data['Price'].values.reshape(-1, 1), lookback)

# Création et entraînement du modèle LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(lookback, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_Y, epochs=100, batch_size=1, verbose=2)

# Prédiction des prix des vols pour l'ensemble de test
test_predict = model.predict(test_X)

# Échelle inverse des prédictions et des vraies valeurs
test_predict = scaler.inverse_transform(test_predict)
test_Y = scaler.inverse_transform(test_Y.reshape(-1, 1))

# Calcul de l'erreur moyenne absolue (MAE) de la prédiction
mae = np.mean(np.abs(test_predict - test_Y))
print('MAE:', mae)
